In [1]:
import os 
import sys 

# Run this incase if you have not installed the repo as a package but still
# want to run this notebook

current_dir = os.getcwd()
dir_to_use = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(dir_to_use)

## Text to SQL evaluation using BirdBench

In this example, we are going to learn how to use `text2sql` package to evaluate different models (open and closed source) using one of the latest text to sql benchmark called [Bird Bench](https://bird-bench.github.io/). 

The flow of benchmarking is super simple. Using our api, you can do the following:

- download the benchmark data.
- filter it on several parameters (like: `difficulty`).
- benchmark them with different open/closed source models of your choice.
- See the leaderboard.

You can also extend our API to evaluate text2sql models for your own data. However for that, your format of the data needs to be similar as how it id done for BirdBench. In future we are going to work on this feature too, to make this process more simpler. Now, let's start by importing some libraries. 

In [2]:
from text2sql.eval.dataset.bird import BirdBenchEvalDataset
from text2sql.eval.settings import SQLGeneratorConfig

`BirdBenchEvalDataset` helps you to download and filter / offset the evaluation dataset before you evaluate it with your models. You can also apply your prompt instruction to test it with different prompts. 

`SQLGeneratorConfig` is a configuration which will be used across your benchmarking process. Think of this as the settings for your text to sql generation engine you will be using. There are several arguments however these are some argumets you can tweak:

- `model_name (str)`: Think of this as your experiment name. You can give it any name. However we prefer to give it the name by the name of the model which will be used to evaluate. Example: `gpt-4o-zeroshot` or `experiment-1-gpt-4o-mini`. 

- `use_knowledge (bool)`: When you set this as `True` it will be inserting existing domain knowledge (around the database or tables) inside the prompt.

You can also check out other parameters too, but those are not required to changed from your side. 

In [3]:
config = SQLGeneratorConfig(model_name="gpt-4o-50-examples")
eval_dataset = BirdBenchEvalDataset(config=config)

Now let's load our dataset. By default it will download the whole dev-set of BirdBench dataset. So, in this example, I am going to show you how you can download and offset the number of examples for evaluation and also filter it by difficulty. 

In [4]:
filter_by = ("difficulty", "simple")
offset = 50

dataset = eval_dataset.process_and_filter(
    num_rows=offset, 
    filter_by=filter_by
)

2024-07-29 13:30:40,421 - text2sql-eval - INFO - ./data/eval/ is not empty. Use force=True to re-download and overwrite the contents.


**NOTE**

Argument `filter_by` is a key, value pair. For BirdBench dataset there are only two types of filter which you can apply. Here are those:

- When the key is set to: `difficulty`, the values can be: 
  - simple
  - moderate
  - challenging

- When the key is set to `db_id`, the values can be:
  - european_football_2
  - toxicology
  - california_schools
  - student_club
  - superhero
  - card_games
  - thrombosis_prediction
  - financial
  - codebase_community
  - debit_card_specializing
  - formula_1

Key `db_id` only does the evaluation for a specific database. You can check the contents of the database once the data is download inside the `data/eval/dev_databases` folder. This folder will be present inside the folder you are running the experiments. 

The `dataset` will be a list of dictionary here. Let's see how it looks like:

In [5]:
dataset

[
    {
        "question_id": 0,
        "db_id": "california_schools",
        "question": "What is the highest eligible free rate for K-12 students in the schools in Alameda County?",
        "evidence": "Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`",
        "SQL": "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1",
        "difficulty": "simple",
        "db_path": "./data/eval/dev_databases/california_schools/california_schools.sqlite"
    },
    {
        "question_id": 2,
        "db_id": "california_schools",
        "question": "Please list the zip code of all the charter schools in Fresno County Office of Education.",
        "evidence": "Charter schools refers to `Charter School (Y/N)` = 1 in the table fprm",
        "SQL": "SELECT T2.Zip FROM frpm AS T1 INNER JOIN schools AS T2 ON T1.CDSCode = T2.CDSCode WHERE T

As you can see in the dataset, each of them have a question, the name of the database, a question, an evidence (which acts as additional domain knowledge here), SQL (the actual SQL) and difficulty (which tells how complex the query is) and a db_path which gives the path of the sqlite database. 

Now we can use the information of qestion, evidence to build our prompts. Here is an example

In [6]:
dataset_with_prompt = dataset.apply_prompt(
    apply_knowledge=True, 
    header_prompt=None
)

dataset_with_prompt[0]

{'question_id': 0,
 'db_id': 'california_schools',
 'question': 'What is the highest eligible free rate for K-12 students in the schools in Alameda County?',
 'evidence': 'Eligible free rate for K-12 = `Free Meal Count (K-12)` / `Enrollment (K-12)`',
 'SQL': "SELECT `Free Meal Count (K-12)` / `Enrollment (K-12)` FROM frpm WHERE `County Name` = 'Alameda' ORDER BY (CAST(`Free Meal Count (K-12)` AS REAL) / `Enrollment (K-12)`) DESC LIMIT 1",
 'difficulty': 'simple',
 'db_path': './data/eval/dev_databases/california_schools/california_schools.sqlite',
 'prompt': 'CREATE TABLE frpm\n(\n    CDSCode                                       TEXT not null\n        primary key,\n    `Academic Year`                               TEXT  null,\n    `County Code`                                 TEXT  null,\n    `District Code`                               INTEGER         null,\n    `School Code`                                 TEXT  null,\n    `County Name`                                 TEXT null,\n 

Let's watch the prompt closely here:

In [7]:
print(dataset_with_prompt[0]["prompt"])

CREATE TABLE frpm
(
    CDSCode                                       TEXT not null
        primary key,
    `Academic Year`                               TEXT  null,
    `County Code`                                 TEXT  null,
    `District Code`                               INTEGER         null,
    `School Code`                                 TEXT  null,
    `County Name`                                 TEXT null,
    `District Name`                               TEXT null,
    `School Name`                                 TEXT null,
    `District Type`                               TEXT null,
    `School Type`                                 TEXT null,
    `Educational Option Type`                     TEXT null,
    `NSLP Provision Status`                       TEXT null,
    `Charter School (Y/N)`                        INTEGER    null,
    `Charter School Number`                       TEXT  null,
    `Charter Funding Type`                        TEXT null,
    IRC             

We fetch the full description of the database along with the schema of the table and insert those schema inside the prompt. If you use:

- `apply_knowledge (bool)`: set as True, then it will use the evidence column else not. From experiments, it has been see that using knowledge it is more efficient. 

- `header_prompt (str)`: You can put additional instruction prompt to optimize or also you can put few shot examples from this. 

`dataset` internally is an object of `DataInstance` class, and you can use this to work out few shot examples to put inside header prompt for different tables. We can show that in an another example.

Now, let's evaluate `gpt-4o` by OpenAI for the sample of our dataset. We are going to use Prem AI API for this. You can check out how to get started with Prem AI in our [documentation](https://docs.premai.io/introduction).  

In [8]:
from text2sql.eval.generator.bird.from_api import SQLGeneratorFromAPI
from text2sql.eval.settings import APIConfig

api_key = os.environ.get("PREMAI_API_KEY")

`APIConfig` acts as an config to control your LLM. You need to put your `api_key` and the `model_name` as a required parameter. Additionally you can also put other parameters like `temperature`, `max_tokens` etc to tweak generations. 

`SQLGeneratorFromAPI` acts as our engine that generates responses and appends those generations inside our initial data. We will used the data (containing the llm generations) to furthur evaluate them. In this step we are only going to generate results.

In [33]:
api_config = api_config = APIConfig(
    api_key=api_key, 
    temperature=0.1, 
    max_tokens=256,
    model_name="gpt-4o"
)

client_gpt4o = SQLGeneratorFromAPI(
    generator_config=config,
    engine_config=api_config
)

data_with_gen = client_gpt4o.generate_and_save_results(
    data=dataset_with_prompt, force=False
)

100%|██████████| 50/50 [02:19<00:00,  2.79s/it]
2024-07-29 13:10:11,526 - text2sql-eval - INFO - all responses written to ./experiments/eval/prem_gpt-4o-5-examples/predict_dev.json
2024-07-29 13:10:11,526 - text2sql-eval - INFO - all responses written to ./experiments/eval/prem_gpt-4o-5-examples/predict_dev.json


To get our results, we call the function `generate_and_save_results` which has two parameters:

- `data`: The dataset that we want to pass for evaluation. 
- `force`: Whether to force generation. By default it stays to False. 
- 

This function first checks if there is any folder inside `eval/experiments/<config.model_name>` (in our case: `eval/experiments/prem_gpt-4o-5-examples`) and if force is False, then it will return the answers without calling the API (to reduce cost) else, it will call the API. If `force` is True then it will invoke generation and override the existing generation results.

Great, we have done the generation, now let's move to evaluation. 

In [34]:
from text2sql.eval.executor.bird.acc import BirdExecutorAcc
from text2sql.eval.executor.bird.ves import BirdExecutorVES

For evaluation, we will use two classes, `BirdExecutorAcc` and `BirdExecutorVES`. These two are different metric that are popularly used to evaluate text to SQL tasks. 

The first one is very straight forward accuracy metric. The second one VES or Valid Efficiency Score is a score which returns 0 if the predicted SQL gives wrong result else it gives the relative efficiency (time required to execute the predicted SQL over the actual SQL). VES is super important when it comes to determine, how optimized SQL does our LLM generates. 

We start off by instantiating both of the metric.

In [35]:
acc = BirdExecutorAcc(generator_config=config)
ves = BirdExecutorVES(generator_config=config)

Now we simply execute them. 

**PLEASE NOTE**

If you have used any kind of filter (which is based on difficulty) before to get the dataset, then provide the filter while you do the execution, else it can be None. Here is how we do it. If you have used filter based on `db_id` no need to put anything inside `filter_used` argument. 

In [36]:
# filter_used: None if filter_by is None or no filter is used 
acc_res = acc.execute(model_responses=data_with_gen, filter_used=filter_by)

=====================   ACCURACY    =====================
+-------------+-------------------+-------------------+
| Category    |   num_correct (%) |   total questions |
+=============+===================+===================+
| simple      |                46 |                50 |
+-------------+-------------------+-------------------+
| overall     |                46 |                50 |
+-------------+-------------------+-------------------+
| moderate    |                 0 |                 0 |
+-------------+-------------------+-------------------+
| challenging |                 0 |                 0 |
+-------------+-------------------+-------------------+


**NOTE**

Since we have used filters here (based on difficulty) so it only shows the score of that difficulty here. In the next example, I am also going to show one small example on how to do full evaluation (without using filter for a small subset of the data). 

The `num_correct` is in percentage out of the `total questions`. So `gpt-4o` does give around 23 correct answers out of 50. 

In [38]:
ves_res = ves.execute(model_responses=data_with_gen, filter_used=filter_by)

=====================   VES    =====================
+-------------+-----------+-------------------+
| Category    |   VES (%) |   total questions |
+=============+===========+===================+
| simple      |   46.6103 |                50 |
+-------------+-----------+-------------------+
| moderate    |    0      |                 0 |
+-------------+-----------+-------------------+
| challenging |    0      |                 0 |
+-------------+-----------+-------------------+
| overall     |   46.6103 |                50 |
+-------------+-----------+-------------------+


The same goes for valid efficiency score too. You will see a different score than accuracy because of the formula we used.  

### Doing full evaluation for gpt-4o

In [10]:
from text2sql.eval.dataset.bird import BirdBenchEvalDataset
from text2sql.eval.settings import SQLGeneratorConfig, APIConfig
from text2sql.eval.generator.bird.from_api import SQLGeneratorFromAPI
from text2sql.eval.executor.bird.acc import BirdExecutorAcc
from text2sql.eval.executor.bird.ves import BirdExecutorVES


config = SQLGeneratorConfig(model_name="gpt-4o-full-eval")
eval_dataset = BirdBenchEvalDataset(config=config)

dataset = eval_dataset.process_and_filter().apply_prompt(apply_knowledge=True)
api_config = api_config = APIConfig(
    api_key=api_key, 
    temperature=0.1, 
    max_tokens=256,
    model_name="gpt-4o"
)

client_gpt4o = SQLGeneratorFromAPI(
    generator_config=config,
    engine_config=api_config
)

data_with_gen = client_gpt4o.generate_and_save_results(
    data=dataset, force=False
)

acc = BirdExecutorAcc(generator_config=config)
ves = BirdExecutorVES(generator_config=config)

acc_res = acc.execute(model_responses=data_with_gen)
ves_res = ves.execute(model_responses=data_with_gen)

2024-07-29 13:36:25,006 - text2sql-eval - INFO - ./data/eval/ is not empty. Use force=True to re-download and overwrite the contents.
2024-07-29 13:36:25,006 - text2sql-eval - INFO - ./data/eval/ is not empty. Use force=True to re-download and overwrite the contents.
 18%|█▊        | 273/1534 [13:16<1:02:22,  2.97s/it]

We are very quickly iterating our API for a full suite of developing, tuning and evaluating text to sql pipelines. In our future set of releases we are going to add support for doing fine-tuning on different datasets and we want to evaluate and see how small language models can perform and can they beat the current score of closed source models like gpt-4o.

We are also going to add more tutorials on how to use our evaluator in your existing pipelines or with langchain / llama-index or DSPy etc. So stay tuned. 